In [1]:
import yfinance as yf
import pandas as pd
import plotly.express as px

### Task 1 - SMA Crossover Strategy

In [2]:
df = yf.download("TCS.NS", period="6mo", multi_level_index=False)

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


In [3]:
# Calculating the 5-day and 20-day Simple Moving Averages (SMA)
df["5_Day_SMA"] = df["Close"].rolling(window=5).mean()
df["20_Day_SMA"] = df["Close"].rolling(window=20).mean()

# Determinig the trend
# 1 if 5-day SMA > 20-day SMA, -1 if 5-day SMA < 20-day SMA
df["Trend"] = 0
df.loc[df["5_Day_SMA"] > df["20_Day_SMA"], "Trend"] = 1  # 1 indicates a recent uptrend
df.loc[df["5_Day_SMA"] < df["20_Day_SMA"], "Trend"] = -1  # -1 indicates a recent downtrend

# Calculating the change in trend to identify crossover points
df["Shift"] = df["Trend"].diff()

# Buy signal: when trend changes from -1 to 1 (bullish crossover)
df["Buy_Signals"] = df["Shift"] == 2
# Sell signal: when trend changes from 1 to -1 (bearish crossover)
df["Sell_Signals"] = df["Shift"] == -2

In [4]:
# Plotting the Close price, 5-day SMA, and 20-day SMA
fig = px.line(
    df,
    x=df.index,
    y=["Close", "5_Day_SMA", "20_Day_SMA"],
    labels={"variable": "Legend", "value": "Price","5_Day_SMA": "5 Day SMA"},
    title="TCS - SMA Crossover Strategy",
    height=600,
    width=1200
)

# Adding green triangle markers for buy signals
fig.add_scatter(
    x=df[df["Buy_Signals"]].index,
    y=df.loc[df["Buy_Signals"], "5_Day_SMA"],
    mode="markers",
    marker=dict(symbol="triangle-up", size=20, color="green"),
    name="Buy",
)

# Adding red triangle markers for sell signals
fig.add_scatter(
    x=df[df["Sell_Signals"]].index,
    y=df.loc[df["Sell_Signals"], "20_Day_SMA"],
    mode="markers",
    marker=dict(symbol="triangle-down", size=20, color="red"),
    name="Sell",
)
fig

### Task 2 - Volatility Analysis

In [5]:
df = yf.download("INFY.NS", period="6mo", multi_level_index=False)

[*********************100%***********************]  1 of 1 completed


In [6]:
# Forward-filling missing data with previous day's data
df = df.ffill()

# Calculating daily percent returns
df["Returns"] = df["Close"].pct_change() * 100

# Calculating the 7 day rolling mean and standard deviation
df["Rolling_Returns"] = df["Returns"].rolling(7).mean()
df["Rolling_Volatility"] = df["Returns"].rolling(7).std()

# Upper and lower bounds for the Volatility Bands (Mean +/- Std Dev)
df["Upper"] = df["Rolling_Returns"] + df["Rolling_Volatility"]
df["Lower"] = df["Rolling_Returns"] - df["Rolling_Volatility"]

In [7]:
# Plotting Daily Returns, Rolling Returns, and Volatility Bands
fig = px.line(
    df,
    x=df.index,
    y=["Returns", "Rolling_Returns", "Upper", "Lower"],
    width=1600,
    color_discrete_map={
        "Returns": "gray",
        "Rolling_Returns": "blue",
        "Upper": "rgba(0,0,0,0)",   # invisible line for upper
        "Lower": "rgba(0,0,0,0)"    # invisible line for lower
    },
    line_dash_map={
        "Returns": "dotted",
        "Rolling_Returns": "solid",
        "Upper": "solid",
        "Lower": "solid"
    },
    labels={
        "value": "Returns",
        "variable": "Legend"
    }
)

# Filling between Upper and Lower Bands using add_traces
fig.add_traces([
    dict(
        type="scatter",
        x=df.index,
        y=df["Upper"],
        mode="lines",
        line=dict(width=0),
        showlegend=False,
        hoverinfo='skip'
    ),
    dict(
        type="scatter",
        x=df.index,
        y=df["Lower"],
        mode="lines",
        fill='tonexty',
        fillcolor='rgba(0,176,246,0.2)',  # light blue fill
        line=dict(width=0),
        name="Volatility Band",
        hoverinfo='skip'
    )
])

# Making Returns line less prominent
fig.update_traces(
    selector=dict(name="Returns"),
    line=dict(color="gray", width=1.5, dash="dot"),
    opacity=1
)

# Making Rolling Returns line more prominent
fig.update_traces(
    selector=dict(name="Rolling_Returns"),
    line=dict(color="blue", width=2)
)

fig.update_layout(
    title="Returns, Rolling Returns, and Volatility Band",
    yaxis_title="Returns",
    xaxis_title="Date"
)

fig

### Task 3 - Portfolio Tracker

In [8]:
portfolio = {
    "INFY.NS": 15,
    "TCS.NS": 10,
    "RELIANCE.NS": 12
}
# Downloading the last 30 days of closing prices for the stocks in the portfolio
df = yf.download(list(portfolio.keys()), period="30d", auto_adjust=True)["Close"]

# Calculating the total portfolio value over time
ttl_portf = (
    df
    .ffill()  # Forward-filling missing values in the DataFrame to handle any gaps in data.
    .mul(pd.Series(portfolio), axis="columns")  # Multiplying each stock's closing price by the number of shares held (from the portfolio dictionary).
    .sum(axis="columns")  # Summing the values across all stocks for each date to get the total portfolio value.
)
ttl_portf

[*********************100%***********************]  3 of 3 completed


,0
Date,
2025-04-02,73705.550171
2025-04-03,71463.398438
2025-04-04,69225.148804
2025-04-07,67680.149902
2025-04-08,68547.899658
2025-04-09,67745.699951
2025-04-11,68084.148682
2025-04-15,68753.199219
2025-04-16,68806.100708


In [9]:
# Plotting the total portfolio value over time as a line chart
fig = px.line(
    x=ttl_portf.index,
    y=ttl_portf,
    title="Total Portfolio over Time",
    labels={"x": "Date", "y": "Total Portfolio"},
    template="plotly",
    width=1600,
    height=600,
    markers=True
    )

# Highlight the latest value with a marker and annotation
fig.add_scatter(
    x=[ttl_portf.index[-1]],
    y=[ttl_portf.iloc[-1]],
    mode="markers+text",
    marker=dict(color="red", size=12),
    text=[f"₹{ttl_portf.iloc[-1]:,.2f}"],
    textposition="top right",
    name="Latest Value"
)

fig.update_layout(
    title={
        "text": "Total Portfolio Value Over Time",
        "x": 0.5,
        "xanchor": "center"
    },
    xaxis=dict(showgrid=True, gridcolor="lightgray"),
    yaxis=dict(showgrid=True, gridcolor="lightgray"),
    legend=dict(title="Legend", orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
)

fig

In [10]:

print(f"Latest Total Value of Portfolio on {ttl_portf.index[-1]:%d %b, %Y} is ₹{ttl_portf.iloc[-1]:.2f}")

Latest Total Value of Portfolio on 19 May, 2025 is ₹75874.60


### Task 4 - Stock Scanner

In [11]:
# List of stock tickers to analyze
STOCKS = [
    "RELIANCE.NS",
    "TCS.NS",
    "INFY.NS",
    "HDFCBANK.NS",
    "ICICIBANK.NS",
    "ITC.NS",
    "LT.NS",
    "SBIN.NS",
    "AXISBANK.NS",
    "KOTAKBANK.NS",
    "WIPRO.NS",
    "HCLTECH.NS",
    "HINDUNILVR.NS",
    "BAJFINANCE.NS",
    "SUNPHARMA.NS",
    "NESTLEIND.NS",
    "MARUTI.NS",
    "ADANIENT.NS",
    "BHARTIARTL.NS",
    "BAJAJFINSV.NS",
]
# Downloading 1 month of adjusted close prices for the selected stocks
df = yf.download(STOCKS, period="1mo", auto_adjust=True)["Close"]

[*********************100%***********************]  20 of 20 completed


In [12]:
# Calculate percent returns over 1 month for each stock
returns = ((df.iloc[-1] - df.iloc[0]) / (df.iloc[0])) * 100

# Sort returns in descending order
returns_sorted = returns.sort_values(ascending=False)

# Select top 5 gainers and bottom 5 losers
outliers = pd.concat([returns_sorted.head(5), returns_sorted.tail(5)])
outliers

,0
Ticker,
HCLTECH.NS,12.593213
RELIANCE.NS,11.215751
MARUTI.NS,10.591741
LT.NS,9.745464
INFY.NS,7.498280
SUNPHARMA.NS,-0.842459
AXISBANK.NS,-1.651672
BAJAJFINSV.NS,-3.099594
BHARTIARTL.NS,-3.456513


In [13]:
# Plotting bar chart for top 5 gainers and losers
fig = px.bar(
    x=outliers.index,
    y=outliers.values,
    title="Top 5 Gainers and Losers (1 Month)",
    labels={
        "value": "Percent Change",
        "x": "Stock",
        "y": "Percent Change",
    },
    color=outliers.values,
    color_continuous_scale="RdYlGn",
    text=outliers.apply(lambda x: f"{x:.2f}%"),
    height=600,
    width=1200,
)

fig

In [14]:
# Saving the outliers (gainers and losers) to a CSV file
outliers.to_csv("gainers_losers.csv",float_format=lambda x: f"{x:.2f}%", header=["Percent Change"], index_label="Stock Ticker")